**Dự án này được nấu bởi [Phạm Huỳnh Anh](https://github.com/PhamHuynhAnh16)**

# **Vui lòng không sử dụng dự án với bất kỳ mục đích nào vi phạm đạo đức, pháp luật, hoặc gây tổn hại đến cá nhân, tổ chức...**

# **Trong trường hợp người sử dụng không tuân thủ các điều khoản hoặc vi phạm, tôi sẽ không chịu trách nhiệm về bất kỳ khiếu nại, thiệt hại, hay trách nhiệm pháp lý nào, dù là trong hợp đồng, do sơ suất, hay các lý do khác, phát sinh từ, ngoài, hoặc liên quan đến phần mềm, việc sử dụng phần mềm hoặc các giao dịch khác liên quan đến phần mềm.**

In [ ]:
#@title **🌏 Cài đặt**
from ipywidgets import Button
from IPython.display import clear_output

print("👩🏻‍💻 Cài đặt...")

!git clone https://github.com/PhamHuynhAnh16/Vietnamese-RVC /content/Vietnamese_RVC > /dev/null 2>&1
!pip install -r /content/Vietnamese_RVC/requirements.txt --no-cache-dir -q > /dev/null 2>&1

#@markdown **💻 Cài đặt sẽ mất khoảng 2 phút để hoàn tất!**

clear_output()
Button(description="\u2714 Hoàn Tất!", button_style="success")

In [ ]:
#@title **📱 Mở giao diện sử dụng**
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%cd /content/Vietnamese_RVC
#@markdown **Để trải nghiệm hết tính năng hãy dùng giao diện:) Còn muốn đơn giản thì không dùng giao diện**

#@markdown **Nếu biết có thể sử dụng biểu đồ để kiểm tra huấn luyện quá sức 👍**
su_dung_bieu_do = False #@param {type:"boolean"}

if su_dung_bieu_do:
    %load_ext tensorboard
    %tensorboard --logdir ./assets/logs --port=6870

!python main/app/app.py --share

# **Tùy chỉnh thêm 🧰**

In [ ]:
#@title **Chỉnh sửa độ chính xác**
import json

from ipywidgets import Button
from IPython.display import clear_output, display

%cd /content/Vietnamese_RVC

from main.configs.config import Config

muc_do = "fp32" #@param ["fp16", "fp32"]

config = Config()
configs_json = os.path.join("main", "configs", "config.json")

def change_fp(fp):
    fp16 = fp == "fp16"

    if fp16 and config.device == "cpu": print("CPU không hỗ trợ fp16")
    else:
        configs = json.load(open(configs_json, "r"))
        configs["fp16"] = config.is_half = fp16

        with open(configs_json, "w") as f:
            json.dump(configs, f, indent=4)

change_fp(muc_do)
clear_output()
display(Button(description="\u2714 Hoàn Tất!", button_style="success"))

In [ ]:
#@title **Kết nối hoặc ngắt kết nối với drive ☁**
import os

from ipywidgets import Button
from google.colab import drive
from IPython.display import clear_output, display

if os.path.exists("/content/drive"):
    print("🔗 Ngắt kết nối với drive...")
    try:
        drive.flush_and_unmount()
        clear_output()
        display(Button(description="\u2714 Hoàn Tất!", button_style="success"))
    except Exception as e:
        raise ValueError(f'Đã xảy ra lỗi trong quá trình ngắt kết nối drive: {e}')
else:
    print('🔗 Kết nối với drive...')
    try:
        drive.mount('/content/drive')
        clear_output()
        display(Button(description="\u2714 Hoàn Tất!", button_style="success"))
    except Exception as e:
        raise ValueError(f'Đã xảy ra lỗi trong quá trình kết nối drive: {e}')

In [ ]:
#@title **Khởi động hoặc ngừng sao lưu 🛠**
import os
import time
import threading
import subprocess

from ipywidgets import Button
from google.colab import drive
from IPython.display import clear_output

logs_folder, weights_folder, audios_folder = '/content/drive/MyDrive/model/logs', '/content/drive/MyDrive/model/weights', '/content/drive/MyDrive/audios'

#@markdown **Nếu không tích vào ô nào thì sẽ ngừng kết nối ở phần đó**
khoi_dong_sao_luu_mo_hinh = False #@param {"type":"boolean"}
khoi_dong_sao_luu_am_thanh = False #@param {"type":"boolean"}
#@markdown **Đồng bộ là sẽ đồng bộ các thư mục sao lưu lại, việc thư mục bị xóa mất 1 tệp thì ở thư mục sao lưu tệp đó cũng sẽ bị xóa**
dong_bo_thu_muc = False #@param {"type":"boolean"}

%cd /content/Vietnamese_RVC

class Channel:
    def __init__(self, source, destination, sync_deletions=False, every=60, exclude = None):
        self.source = source
        self.destination = destination
        self.event = threading.Event()
        self.syncing_thread = threading.Thread(target=self._sync, args=())
        self.sync_deletions = sync_deletions
        self.every = every

        if not exclude: exclude = []
        if isinstance(exclude, str): exclude = [exclude]

        self.exclude = exclude
        self.command = ['rsync', '-aP']

    def alive(self):
        if self.syncing_thread.is_alive(): return True
        else: return False

    def _sync(self):
        command = self.command

        for exclusion in self.exclude:
            command.append(f'--exclude={exclusion}')

        command.extend([f'{self.source}/', f'{self.destination}/'])

        if self.sync_deletions: command.append('--delete')

        while not self.event.is_set():
            subprocess.run(command)
            time.sleep(self.every)

    def copy(self):
        command = self.command

        for exclusion in self.exclude:
            command.append(f'--exclude={exclusion}')

        command.extend([f'{self.source}/', f'{self.destination}/'])

        if self.sync_deletions: command.append('--delete')
        subprocess.run(command)

        return True

    def start(self):
        if self.syncing_thread.is_alive():
            self.event.set()
            self.syncing_thread.join()

        if self.event.is_set(): self.event.clear()
        if self.syncing_thread._started.is_set(): self.syncing_thread = threading.Thread(target=self._sync, args=())

        self.syncing_thread.start()
        return self.alive()

    def stop(self):
        if self.alive():
            self.event.set()
            self.syncing_thread.join()

            while self.alive():
                if not self.alive(): break

        return not self.alive()

if not "logs_backup" in locals(): logs_backup = Channel("/content/Vietnamese_RVC/assets/logs", logs_folder, sync_deletions=dong_bo_thu_muc, every=40, exclude="mute")
if not "weights_backup" in locals(): weights_backup = Channel("/content/Vietnamese_RVC/assets/weights", weights_folder, sync_deletions=dong_bo_thu_muc, every=40)
if not "audio_backup" in locals(): audio_backup = Channel("/content/Vietnamese_RVC/audios", audios_folder, sync_deletions=dong_bo_thu_muc, every=40)

logs_backup.stop(); weights_backup.stop(); audio_backup.stop()

if os.path.exists('/content/drive/MyDrive'):
    if khoi_dong_sao_luu_mo_hinh:
        if not os.path.exists(logs_folder): os.makedirs(logs_folder)
        if not os.path.exists(weights_folder): os.makedirs(weights_folder)

        logs_backup.start(); weights_backup.start()
    else: logs_backup.stop(); weights_backup.stop()

    if khoi_dong_sao_luu_am_thanh:
        if not os.path.exists(audios_folder): os.makedirs(audios_folder)
        audio_backup.start()
    else: audio_backup.stop()
else:
    try:
        drive.mount('/content/drive')
    except Exception as e:
        raise ValueError(f'Đã xảy ra lỗi trong quá trình kết nối drive: {e}')

    if khoi_dong_sao_luu_mo_hinh:
        if not os.path.exists(logs_folder): os.makedirs(logs_folder)
        if not os.path.exists(weights_folder): os.makedirs(weights_folder)

        logs_backup.start(); weights_backup.start()
    else: logs_backup.stop(); weights_backup.stop()

    if khoi_dong_sao_luu_am_thanh:
        if not os.path.exists(audios_folder): os.makedirs(audios_folder)
        audio_backup.start()
    else: audio_backup.stop()

clear_output()
Button(description="\u2714 Hoàn Tất!", button_style="success")

In [ ]:
#@title **♻️ Khởi động dọn rác**
import os
import time
import threading

from ipywidgets import Button
from google.colab import auth, drive
from IPython.display import clear_output

try:
    from googleapiclient.discovery import build
except:
    os.system("pip install google-api-python-client")
    from googleapiclient.discovery import build

class Clean:
    def __init__(self, every=60):
        self.service = build('drive', 'v3')
        self.every = every
        self.trash_cleanup_thread = None

    def delete(self):
        page_token = None

        while 1:
            response = self.service.files().list(q="trashed=true", spaces='drive', fields="nextPageToken, files(id, name)", pageToken=page_token).execute()

            for file in response.get('files', []):
                if file['name'].startswith("G_") and file['name'].endswith(".pth") or file['name'].startswith("D_") and file['name'].endswith(".pth"):
                    try:
                        self.service.files().delete(fileId=file['id']).execute()
                    except Exception as e:
                        raise RuntimeError(e)

            page_token = response.get('nextPageToken', None)
            if page_token is None: break

    def clean(self):
        while 1:
            self.delete()
            time.sleep(self.every)

    def start(self):
        self.trash_cleanup_thread = threading.Thread(target=self.clean)
        self.trash_cleanup_thread.daemon = True
        self.trash_cleanup_thread.start()

    def stop(self):
        if self.trash_cleanup_thread: self.trash_cleanup_thread.join()

%cd /content/Vietnamese_RVC

#@markdown **Sử dụng khi huấn luyện sẽ dọn bớt các tệp tin D, G trong thùng rác google drive**
khoi_dong_don_rac = False #@param {"type":"boolean"}

if khoi_dong_don_rac:
    if os.path.exists('/content/drive/MyDrive'):
        auth.authenticate_user()
    else:
        try:
            drive.mount('/content/drive')
        except Exception as e:
            raise ValueError(f'Đã xảy ra lỗi trong quá trình kết nối drive: {e}')

        auth.authenticate_user()
    Clean(every=40).start()
else: Clean().stop()

clear_output()
Button(description="\u2714 Hoàn Tất!", button_style="success")

In [ ]:
#@title **Tải dữ liệu sao lưu từ drive 📂**
import os
import shutil
from ipywidgets import Button
from IPython.display import clear_output, display

logs_folder, weights_folder, audios_folder ='/content/drive/MyDrive/model/logs', '/content/drive/MyDrive/model/weights', '/content/drive/MyDrive/audios'

#@markdown **Tải các mô hình huấn luyện để tiếp tục huấn luyện**
tai_mo_hinh = False # @param {"type":"boolean"}
#@markdown **Tải các âm thanh được sao lưu để tiếp tục sử dụng**
tai_am_thanh = False # @param {"type":"boolean"}

if os.path.exists("/content/drive/MyDrive"):
  if tai_mo_hinh:
    if len(os.listdir(logs_folder)) < 1 or len(os.listdir(weights_folder)) < 1: print("Không tìm thấy dữ Liệu")
    else:
      if os.path.exists("/content/drive/MyDrive/model"):
        shutil.copytree(logs_folder, "/content/Vietnamese_RVC/assets/logs", dirs_exist_ok=True)
        shutil.copytree(weights_folder, "/content/Vietnamese_RVC/assets/weights", dirs_exist_ok=True)

        clear_output()
      else: print("Không tìm thấy dữ liệu mô hình")
  if tai_am_thanh:
    if len(os.listdir(audios_folder)) < 1: print("Không tìm thấy dữ Liệu")
    else:
      if os.path.exists("/content/drive/MyDrive/audios"):
        shutil.copytree(audios_folder, "/content/Vietnamese_RVC/audios", dirs_exist_ok=True)
        clear_output()
      else: print("Không tìm thấy dữ liệu âm thanh")
else: print("Google drive không được kết nối")

display(Button(description="\u2714 Hoàn Tất!", button_style="success"))